In [ ]:
!unzip Gd_fps.zip

In [30]:
import pandas as pd
import numpy as np
import os
import random
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

In [31]:
def seed_everything(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_everything(seed=42)

In [32]:
def load_datasets(path: str):
    files = [file for file in os.listdir(path) if file.endswith('.csv')]
    datasets = {}
    for file in files:
        df = pd.read_csv(os.path.join(path, file))
        datasets[file] = df
    return datasets

In [33]:
datasets = load_datasets("Gd_fps")

In [34]:
from sklearn.feature_selection import VarianceThreshold


def filter_var(df: pd.DataFrame) -> tuple[np.ndarray, np.ndarray]:
    feature_cols = [c for c in df.columns if c != "lgK"]
    X = df[feature_cols].astype(np.float32).values
    y = df["lgK"].values
    vt = VarianceThreshold(threshold=0.01)  # remove features with <1% variance
    X = vt.fit_transform(X)
    print(f"Reduced from {len(feature_cols)} to {X.shape[1]} features")
    return X, y

In [35]:
from fedot import Fedot

def init_fedot(problem='regression', timeout=10., n_jobs=-1, logging_level=50, seed=42):
    model = Fedot(
        problem=problem, 
        timeout=timeout, 
        n_jobs=n_jobs, 
        logging_level=logging_level,
        seed=seed
    )
    return model

In [36]:
def cross_validate(X, y, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    cv_results = {}
    for fold, (train_index, test_index) in enumerate(kf.split(X)):
        model = init_fedot(
            problem=problem, 
            timeout=timeout, 
            n_jobs=n_jobs, 
            logging_level=logging_level, 
            seed=seed
        )
        print(f"Fold {fold}")
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        rmse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        cv_results[fold] = {"rmse": rmse, "r2": r2}
        print(f"  RMSE: {rmse}, R2: {r2}")
    return cv_results

In [37]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, KFold


def run_fedot(X, y):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    model = init_fedot(
        problem=problem, 
        timeout=timeout, 
        n_jobs=n_jobs, 
        logging_level=logging_level,
        seed=seed
    )
    cv_scores = cross_validate(X, y)
    results = {"cv_scores": cv_scores}
    print("CV R2 scores:", cv_scores)
    
    pipeline = model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    test_rmse = mean_squared_error(y_test, y_pred)
    test_r2 = r2_score(y_test, y_pred)
    results.update({"test_rmse": test_rmse, "test_r2": test_r2})
    results.update({"pipeline": pipeline})
    print("Test RMSE:", test_rmse)
    print("Test R2:", test_r2)
    
    return results

In [38]:
problem='regression'
timeout = 10.
n_jobs = -1
logging_level = 50
seed = 42

In [39]:
all_results = {}

In [40]:
for dataset_name in datasets:
    df = datasets[dataset_name]
    print(f"Running FEDOT on {dataset_name}")
    X, y = filter_var(df)
    results = run_fedot(X, y)
    all_results[dataset_name] = results

Running FEDOT on Gd_ctopo_fp_cmplx.csv
Reduced from 2048 to 267 features
Fold 0
2025-08-29 16:39:06,201 - Topological features operation requires extra dependencies for time series forecasting, which are not installed. It can infuence the performance. Please install it by 'pip install fedot[extra]'


Generations:   0%|          | 13/10000 [05:46<73:53:55, 26.64s/gen]


  0%|          | 56/100000 [04:09<123:48:11,  4.46s/trial, best loss: 3.99002383725207]  
  RMSE: 14.265312214594873, R2: 0.26467919176069066
Fold 1


Generations:   0%|          | 9/10000 [06:19<116:59:43, 42.16s/gen]


  0%|          | 51/100000 [03:36<118:00:30,  4.25s/trial, best loss: 3.851295053746971] 
  RMSE: 13.249909432223866, R2: 0.22811645123127577
Fold 2


...

  0%|          | 170/100000 [04:00<39:09:33,  1.41s/trial, best loss: 3.9143723367143237]
  RMSE: 14.921490533672811, R2: 0.2717758232781351
Fold 4


Generations:   0%|          | 30/10000 [05:51<32:26:18, 11.71s/gen]


  0%|          | 58/100000 [04:05<117:24:35,  4.23s/trial, best loss: 3.88440351249031]  
  RMSE: 20.168187951517563, R2: 0.15772263374474615
CV R2 scores: {0: {'rmse': 15.950044075925609, 'r2': 0.17783788220471797}, 1: {'rmse': 13.767887026926628, 'r2': 0.1979412725987717}, 2: {'rmse': 17.44370262094669, 'r2': 0.24538561450967955}, 3: {'rmse': 14.921490533672811, 'r2': 0.2717758232781351}, 4: {'rmse': 20.168187951517563, 'r2': 0.15772263374474615}}


Generations:   0%|          | 50/10000 [04:47<15:52:17,  5.74s/gen]


  0%|          | 62/100000 [05:10<139:01:08,  5.01s/trial, best loss: 4.033161386245383]
Test RMSE: 14.624205847861937
Test R2: 0.2461796347572428


In [41]:
all_results

{'Gd_ctopo_fp_cmplx.csv': {'cv_scores': {0: {'rmse': 14.265312214594873,
    'r2': 0.26467919176069066},
   1: {'rmse': 13.249909432223866, 'r2': 0.22811645123127577},
   2: {'rmse': 15.353125673775937, 'r2': 0.3358239504863032},
   3: {'rmse': 11.960099436128054, 'r2': 0.4163027114663066},
   4: {'rmse': 23.422433459547438, 'r2': 0.021816654375630296}},
  'test_rmse': 14.395997764554659,
  'test_r2': 0.2579428650139608,
  'pipeline': {'depth': 1, 'length': 1, 'nodes': [rfr]}},
 'Gd_ctopo_fp_cmplx_da.csv': {'cv_scores': {0: {'rmse': 14.781019852960942,
    'r2': 0.23809648878482792},
   1: {'rmse': 15.593622520663779, 'r2': 0.091581663182006},
   2: {'rmse': 14.990969795267032, 'r2': 0.35149080984817604},
   3: {'rmse': 10.725213185548116, 'r2': 0.47656974853906864},
   4: {'rmse': 26.202524840441615, 'r2': -0.09428738292695149}},
  'test_rmse': 15.009554563153035,
  'test_r2': 0.22631642219524017,
  'pipeline': {'depth': 2, 'length': 2, 'nodes': [rfr, pca]}},
 'Gd_ctopo_fp_cmplx_da_bo

In [46]:
for dataset_name in all_results:
    all_results[dataset_name]["pipeline"].save(f"pipelines/{dataset_name}.json", create_subdir=True, is_datetime_in_path=True)
    all_results[dataset_name].pop("pipeline")

In [48]:
import json

with open("automl_results.json", "w") as f:
    json.dump(all_results, f)